In [5]:
import tensorflow as tf
import time
from tensorflow.examples.tutorials.mnist import input_data

In [3]:
# Creates a graph.
with tf.device('/gpu:0'):
  a = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[2, 3], name='a')
  b = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[3, 2], name='b')
c = tf.matmul(a, b)
# Creates a session with log_device_placement set to True.
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))
# Runs the op.
print(sess.run(c))

[[22. 28.]
 [49. 64.]]


In [7]:
mnist = input_data.read_data_sets("MINIST_data",one_hot=True)

batch_size = 50

n_batch = mnist.train.num_examples // batch_size

with tf.device('/gpu:0'):
    #placeholder
    x = tf.placeholder(tf.float32,[None,784])
    y = tf.placeholder(tf.float32,[None,10])

    lr = tf.Variable(0.001,dtype='float32')
    #控制工作神经元比例
    keep_prob = tf.placeholder(tf.float32)

    #创建简单神经网络
    w1 = tf.Variable(tf.truncated_normal([784,500],stddev = 0.1))
    b1 = tf.Variable(tf.zeros([500]) + 0.1)
    L1 = tf.nn.tanh(tf.matmul(x,w1) + b1)


    L1_drop = tf.nn.dropout(L1,keep_prob)

    w2 = tf.Variable(tf.truncated_normal([500,300],stddev = 0.1))
    b2 = tf.Variable(tf.zeros([300]) + 0.1)
    L2 = tf.nn.tanh(tf.matmul(L1_drop,w2) + b2)

    L2_drop = tf.nn.dropout(L2,keep_prob)


    #输出
    w4 = tf.Variable(tf.truncated_normal([300,10],stddev = 0.1))
    b4 = tf.Variable(tf.zeros([10]) + 0.1)
    #L4 = tf.nn.tanh(tf.matmul(L3_drop,w4) + b4)


    prediction = tf.nn.softmax(tf.matmul(L2_drop,w4)+b4)


    #二次代价函数
    #loss = tf.reduce_mean(tf.square(y - prediction))
    #交叉熵
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels = y,logits = prediction))

    #train_step = tf.train.GradientDescentOptimizer(0.2).minimize(loss)
    train_step = tf.train.AdamOptimizer(lr).minimize(loss)

    init = tf.global_variables_initializer()

    correct_prediction = tf.equal(tf.argmax(y,1),tf.argmax(prediction,1))

    accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))
start = time.time()
with tf.Session() as sess: # config=tf.ConfigProto(log_device_placement=True) 记录用的是CPU还是GPU
    sess.run(init)
    for epoch in range(20):
        sess.run(tf.assign(lr,0.001*(0.95**epoch)))
        for batch in range(n_batch):
            batch_xs,batch_ys = mnist.train.next_batch(batch_size)
            sess.run(train_step,feed_dict = {x:batch_xs,y:batch_ys,keep_prob:1.0})
        
        test_acc = sess.run(accuracy,feed_dict = {x:mnist.test.images,y:mnist.test.labels,keep_prob:1.0})
        #train_acc = sess.run(accuracy,feed_dict = {x:mnist.train.images,y:mnist.train.labels,keep_prob:1.0})
        
        print("Iter " + str(epoch) + ",Testing accuracy " + str(test_acc))
print('Time:{}'.format(time.time() - start))

Extracting MINIST_data\train-images-idx3-ubyte.gz
Extracting MINIST_data\train-labels-idx1-ubyte.gz
Extracting MINIST_data\t10k-images-idx3-ubyte.gz
Extracting MINIST_data\t10k-labels-idx1-ubyte.gz
Iter 0,Testing accuracy 0.9514
Iter 1,Testing accuracy 0.9624
Iter 2,Testing accuracy 0.97
Iter 3,Testing accuracy 0.9707
Iter 4,Testing accuracy 0.9718
Time:50.041642904281616
